Autoencoders

Autoencoders are powerful tools for learning meaningful features that can reconstruct the original input. They are particularly useful for the task of `Unsupervised Learning` where there are training samples but, but unlike image or word classification tasks, there is no matching set of labels. Learning a mapping from X -> Y is impossible because Y either does not exist or we do not know it. Instead, the goal is to discover some hidden qualities or structure in our data. Datasets without labels can happen for many reasons:
    1. Manually labeling and curating data is time consuming.
    2. Finding labeled data that matches real world use-cases is difficult.
    3. Labeled data can be very expensive.
The list above shows why getting large amounts of unlabeled data is faster and more economical than collecting a matching, labeled dataset. 

This begs the question: if so much of the current Deep Learning wave was built on labeled data, is there still room for large, unlabeled datasets? 

One of the first methods for training large nets was based on Unsupervised Learning. Before  modern initialization schemes (i.e. Glorot and He), Deep Nets were initialized with something called `Unsupervised Pre-Training`. The `Restricted Boltzman Machine` (RBM) was at the heart of this initialization scheme and is similar to a two-layer neural network. The first input layer is called the "visible" layer while the second layer is called the "hidden" layer. The main goal of an RBM is reconstructing its input. It does this by learning a good representation of the input in its hidden layer. This learning process is like finding those hidden qualities or structures in the unlabeled training data. `Deep Belief Nets`, a sort of precursor to the Deep Feedforward Network of today, were initialized in a greedy layer-by-layer fashion with RBMs reconstructing the inputs at each successive layer. 

Reconstructing the original input based on the activation of the "hidden" layer leads to the idea of an `Autoencoder`. While an RBM is usually only two layers deep an Autoencoder will look more like a typical DNN with multiple layers. The goal of an Autoencoder is the same as an RBM: reconstruct the input. Autoencoders usually have a single layer that is much smaller than the others. This `bottleneck` layer forces the network to compress its internal representation into a reduced dimension. The hope is that the intermediate layers, especially the bottleneck layer, distill and capture the most important aspects of the input.

If we want to learn good Autoencoder features that can reconstruct the input, then the best performance metric is how well the generated output matches the original input. We can measure this with the reconstruction error defined as `(original - reconstruction)^2`. Autoencoders minimize this error with backpropagation like standard DNNs and not layer-by-layer like an RBM. It is also worth mentioning that the bottleneck activations can be used as features for other networks and models in a variety of ways. The reconstruction error can be difficult to optimize so it is common to minimize the binary cross entropy instead.  

A popular Autoencoder architecuture is the `Encoder-Decoder`. The Encoder compresses the input down to the bottleneck layer. This can be achieved with a series of dense, convolutional, or recurrent layers. The Decoder is usually a mirrored version of the Encoder that restores the bottleneck to its original input dimensions, although it can also be a completely separate network (i.e. for image captioning and language translation tasks). We will see a simple Encoder-Decoder in the `Python` section of this topic.

Denoising Autoencoders

While the ideas of Autoencoders and Encoder-Decoders are very powerful on the surface, there are many pitfalls to consider. To start with most datasets only capture a small subset of conditions that networks will face in the real world. When a network receives new data in the field with different conditions, its performance will suffer because it has never seen anything like it. On the other hand, if we build a large network then it could vastly overfit the training set and perform badly on new data even if it comes from the same conditions and distribution. 

One of the first ideas for tackling this problem was `Denoising Autoencoders`. Here we distort and corrupt the original input before feeding it into the network. However, when we calculate the cost we still compare the reconstruction to the original, uncorrupted inputs. This ideally forces the network to learn robust features that sift through the noisy input. It was also shown that injecting Gaussian noise into the inputs is similar to L2 (i.e. Energy norm, or weight-decay) regularization [ref]. This further prevents the network from overfitting its training set.   

Variational Autoencoders

Recently a new type of Autoencoders called `Variational Autoencoders` [Kingma ref] are excelling in a variety of tasks. They are related to the well-established field of Bayesian Inference and fall under the rapidly growing umbrella of Bayesian Deep Learning [refs]. 

The heart of Bayesian inference is predicting or finding the best model parameters that maximize a probability we are interested in. This could be the probability of a sentence given a series of words or the probability of a correct image label. Bayesian inference comes from the equation describing the relationship between the likelihood, marginal, and prior probabilities.
[Pic of Bayes' equations]

The quantity on the left, the posterior, is what we really want but is impossible to directly sample from in most realistic applications. The key takeaway is that we have most of the values on the right-hand side of the equation. The likelihood is the probability of our data given the model parameters. The prior represents previous knowledge about the problem and what good parameter values might look like. The marginal represents the overall probability of our data. Next we describe what makes the Variational Autoencoders different from vanilla flavor autoencoders.  

Imagine replacing the bottleneck layer in an autoencoder with two additional latent layers. Now, however, the outputs of the layers have new definitions. They become the statistical means and covariances of a diagonal Gaussian distribution. Instead of reconstructing the input from the bottlneck layer, we instead sample from the latent layer, just as we would from any other probability distribution, and use that "code" to reconstruct the input.     

In Variational Autoencoders, the `reparametrization trick` combines the means and covariances with noise and makes it possible to learn good model parameters with backpropagation instead of updating them manually. While it is difficult to visualize good parameter values in higher dimensions, it turns out that a standard Gaussian prior is a good starting point. We then compare the network's learned means and covariances against the Gaussian prior using the KL Divergence formula. KL is not strictly a measure of distance but does capture the overlap between two distributions very well. When the prior is a standard Gaussian, the KL formula reduces to a simple form: [pic]

The KL divergence is added to the typical cost of the Autoencoder like mean reconstruction error or binary cross-entropy. This makes the network learn good parameters for a statistical model through backpropagation. Learned variational distributions are proving incredibly useful in Deep Learning and add much needed interpretability to DNN models.  

Convolutional Autoencoders

Convolutional autoencoders use, as the name suggests, convolutional layers to learn features that capture the input well. If we use a bottleneck layer with convolutions, how do we go from a smaller input dimension to a larger one? The answer that makes all the math work out is something called fractionally-strided convolutions, or transpose convolutions. To picture them on an image, imagine inserting zeros in between pixel values, in addition to padding. Now we can take a regular convolution over this "up-sampled" sparse pattern and the output of the convolution will match the dimensions we need to restore the layer back to its original dimensions. There is an excellent guide to convolutional arithmetic here: [ref] .

This type of autoencoder can be all-convolutional, meaning they have no max, average, or global pooling layers and downsample with strided convolutions instead. This lets the network learn its own downsampling and upsampling functions which can be ideal for tasks like image reconstruction.